In [ ]:
import pandas as pd
from datasets import Dataset


train_prompts_df = pd.read_json('../fn1.7-train-prompts.jsonl', lines=True)
train_prompts = Dataset.from_pandas(train_prompts_df)


In [ ]:
from unsloth import FastLanguageModel

max_seq_length = 5020
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="llama-3.2-3b-fsp-ft",
    model_name="unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

def formatting_prompts_func_test(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo[:-1], tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

dataset = train_prompts.map(formatting_prompts_func, batched = True,)

test_prompts_df = pd.read_json('../fn1.7-test-prompts.jsonl', lines=True)
test_prompts = Dataset.from_pandas(test_prompts_df)

test_dataset = test_prompts.map(formatting_prompts_func_test, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 20,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
import torch
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
# trainer_stats = trainer.train()
trainer_stats = trainer.train()

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

predictions = []

for i, example in enumerate(test_dataset['messages']):
    inputs = tokenizer.apply_chat_template(
        example[:-1],
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 256, use_cache = True)
    # Decode the generated text, add to predictions, ignore prompt in output
    predictions.append(tokenizer.decode(outputs[0]).split("<|start_header_id|>assistant<|end_header_id|>\n\n")[1])

test_prompts_df['responses'] = predictions
test_prompts_df.to_json('llama-3.1-70B-fsp-preds.jsonl', orient='records', lines=True)

In [ ]:
model.save_pretrained("llama-3.1-70B-fsp") # Local saving
tokenizer.save_pretrained("llama-3.1-70B-fsp")